In [14]:
%matplotlib notebook

In [25]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import gmaps

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [26]:
housing_df = pd.read_csv("Resources/DC_Properties.csv")

In [27]:
housing_df2 = housing_df.drop(["Unnamed: 0", "BLDG_NUM", "NATIONALGRID", "X", "Y"], axis=1)
housing_df2.head()

,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,...,ZIPCODE,LATITUDE,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,QUADRANT
0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,1972,...,20009.0,38.914680,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW
1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,1972,...,20009.0,38.914683,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW
2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,1984,...,20009.0,38.914684,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW
3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,1984,...,20009.0,38.914683,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW
4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,1985,...,20009.0,38.914383,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW


In [28]:
housing_df2['YEAR'] = pd.DatetimeIndex(housing_df2['SALEDATE']).year
housing_df2['MONTH'] = pd.DatetimeIndex(housing_df2['SALEDATE']).month
housing_df2.head()

,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,QUADRANT,YEAR,MONTH
0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,1972,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW,2003.0,11.0
1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,1972,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW,2000.0,8.0
2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,1984,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW,2016.0,6.0
3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,1984,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW,2006.0,7.0
4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,1985,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,NW,NaN,NaN


In [29]:
housing_df2 = housing_df2[housing_df2.PRICE.notnull()]
residential_df = housing_df2.loc[housing_df2["SOURCE"] == "Residential"]
residential_df.count().head()

BATHRM       57900
HF_BATHRM    57900
HEAT         57900
AC           57900
NUM_UNITS    57900
dtype: int64

In [30]:
housing_df3 = residential_df[['BATHRM','LATITUDE','LONGITUDE','BEDRM','HF_BATHRM','YEAR','PRICE']]
housing_df3 = housing_df3[housing_df3.PRICE.notnull()]
housing_df3 = housing_df3[~(housing_df3['YEAR'] < 1992.0)] 
housing_df3 = housing_df3[~(housing_df3['YEAR'] > 2017.0)]
housing_df3.head(20)

,BATHRM,LATITUDE,LONGITUDE,BEDRM,HF_BATHRM,YEAR,PRICE
0,4,38.914680,-77.040832,4,0,2003.0,1095000.0
2,3,38.914684,-77.040678,5,1,2016.0,2100000.0
3,3,38.914683,-77.040629,5,1,2006.0,1602000.0
5,3,38.914331,-77.039715,5,2,2010.0,1950000.0
7,3,38.915408,-77.040129,4,1,2011.0,1050000.0
14,3,38.915038,-77.039716,3,1,2011.0,1325000.0
16,3,38.915018,-77.039844,3,1,2010.0,1240000.0
19,3,38.915019,-77.040138,3,1,2001.0,592250.0
20,3,38.911370,-77.033774,5,1,2016.0,33232.0
22,1,38.911368,-77.033925,3,0,2006.0,907400.0


In [40]:
housing_2bd_2ba = housing_df3[housing_df3['BATHRM'] == 2]
housing_2bd_2ba_df = housing_2bd_2ba[housing_2bd_2ba['BEDRM'] == 2]
housing_2bd_2ba_df2 = housing_2bd_2ba_df[housing_2bd_2ba_df['HF_BATHRM'] == 0]
housing_2bd_2ba_df3 = housing_2bd_2ba_df2[housing_2bd_2ba_df2['YEAR'] == 1995.0]

housing_2bd_2ba_df3

,BATHRM,LATITUDE,LONGITUDE,BEDRM,HF_BATHRM,YEAR,PRICE
10281,2,38.896282,-76.995260,2,0,1995.0,150500.0
16968,2,38.881877,-76.988066,2,0,1995.0,104500.0
17008,2,38.878569,-76.987306,2,0,1995.0,100000.0
17028,2,38.886893,-76.988103,2,0,1995.0,85000.0
17235,2,38.891711,-76.984839,2,0,1995.0,82000.0
28445,2,38.947265,-77.088090,2,0,1995.0,250000.0
29265,2,38.949487,-77.086628,2,0,1995.0,259000.0
35582,2,38.967168,-77.072416,2,0,1995.0,252000.0
47931,2,38.929900,-77.028051,2,0,1995.0,87100.0
55560,2,38.963377,-77.020621,2,0,1995.0,76214.0


In [41]:
# Store latitude and longitude in locations
locations = housing_2bd_2ba_df3[["LATITUDE", "LONGITUDE"]]

# Fill NaN values and convert to float
rating = housing_2bd_2ba_df3["PRICE"].astype(float)

In [47]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=10,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))